In [1]:
from pymongo import MongoClient
from config import *

client = MongoClient()
client = MongoClient(connectionString)
database = client['masters']
database.rates

Collection(Database(MongoClient(host=['masters-shard-00-02.cpq5u.mongodb.net:27017', 'masters-shard-00-00.cpq5u.mongodb.net:27017', 'masters-shard-00-01.cpq5u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-wrhudr-shard-0', tls=True), 'masters'), 'rates')

In [2]:
def get_rates_by_user(user):
    cursor = database.rates.aggregate([
        {
            "$unwind": "$reviews"
        },
        {
            '$match' : {'reviews.user._id' : user}
        },
        {
            '$project': {
                'bookId': '$bookId',
                'rate': '$reviews.rate'
                }
        }
    ])
    return list(cursor)    

def get_books_by_ids(bookIds):
    cursor = database.books.aggregate([
        {
            '$match' : {'_id' : { '$in' : bookIds }}
        },
        {
            '$project': {
                'title': '$title',
                'description': '$description',
                'genres': '$genres.name',
                'author': '$author.name'
                }
        }
    ])
    return list(cursor)

def get_books_not_in_ids(bookIds):
    cursor = database.books.aggregate([
        {
            '$match' : {'_id' : { '$nin' : bookIds }}
        },
        {
            '$project': {
                'title': '$title',
                'description': '$description',
                'genres': '$genres.name',
                'author': '$author.name'
                }
        }
    ])
    return list(cursor)

def get_books():
    cursor = database.books.aggregate([
        {
            '$project': {
                'title': '$title',
                'description': '$description',
                'genres': '$genres.name',
                'author': '$author.name'
                }
        }
    ])
    return list(cursor)

In [46]:
from bson.objectid import ObjectId
import pandas as pd

user_rates = get_rates_by_user(ObjectId("620bb9fef23b1bc78052c5e6"))
user_rates

rates_df = pd.DataFrame(user_rates)
rates_df.drop("_id", axis=1, inplace=True)
bookdIds = list(rates_df.bookId)

rated_books = get_books_by_ids(bookdIds)
rated_books_df = pd.DataFrame(rated_books)

books = get_books_not_in_ids(bookdIds)#[:5]
books_df = pd.DataFrame(books)

all_books = pd.concat([rated_books_df, books_df], axis=0)

main_df = pd.merge(all_books, rates_df, how="left", left_on="_id", right_on="bookId")
main_df.drop("bookId", axis=1, inplace=True)
main_df.genres = [' '.join(map(str, l)) for l in main_df.genres]
# main_df

In [47]:
columns_to_join = ["title", "description", "genres"]
main_df['full_desc'] = main_df[columns_to_join].T.agg(' '.join)
# main_df.assign(combined=main_df.agg(list, axis=1))
main_df.drop(columns_to_join, axis=1, inplace=True)

main_df.sort_index(inplace=True)
main_df

,_id,author,rate,full_desc
0,630602ac073e94de49556747,Hajime Isayama,4.0,Attack on Titan: Volume 13 NO SAFE PLACE LEFT ...
1,630602ac073e94de4955674b,Peter V. Brett,5.0,"The Painted Man (The Demon Cycle, Book 1) The ..."
2,630602ac073e94de4955674d,J.M. Barlog,2.0,God of War: The Official Novelization The nove...
3,630602ac073e94de49556751,Andrzej Sapkowski,4.0,Sword of Destiny: Witcher 2: Tales of the Witc...
4,630602ac073e94de49556781,George R.R. Martin,3.0,A Game of Thrones: The Story Continues Books 1...
...,...,...,...,...
241,630602ac073e94de49556838,Sun Tzu,NaN,The Complete Art of War Collected here in this...
242,630602ac073e94de49556839,Disha Experts,NaN,(FREE SAMPLE) CTET English & Hindi Language 9 ...
243,630602ac073e94de4955683a,Yevgeny Zamyatin,NaN,We Yevgeny Zamyatin's We is set in an urban gl...
244,630602ac073e94de4955683b,Disha Experts,NaN,1 Sample Paper for CBSE Class 10 Science 2020 ...


In [48]:
import string, nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer#, TfidfTransformer

def text_preprocess(sample):
    # lower-casing
    lower_text = sample.lower()

    entry_no_punct = lower_text.translate(str.maketrans('', '', string.punctuation))

    # word-level tokenization
    words = word_tokenize(entry_no_punct)

    # stopwords removal
    stops = stopwords.words('english')
    clean = [word for word in words if word not in stops]

    ps = PorterStemmer()
    # using Porter stemmer to stem our sentence
    stemmed_list = [ps.stem(w) for w in clean]

    return ' '.join(stemmed_list)

In [49]:
main_df["full_desc"] = main_df['full_desc'].apply(text_preprocess)
main_df

,_id,author,rate,full_desc
0,630602ac073e94de49556747,Hajime Isayama,4.0,attack titan volum 13 safe place left great co...
1,630602ac073e94de4955674b,Peter V. Brett,5.0,paint man demon cycl book 1 stun debut fantasi...
2,630602ac073e94de4955674d,J.M. Barlog,2.0,god war offici novel novel highli anticip god ...
3,630602ac073e94de49556751,Andrzej Sapkowski,4.0,sword destini witcher 2 tale witcher geralt wi...
4,630602ac073e94de49556781,George R.R. Martin,3.0,game throne stori continu book 15 game throne ...
...,...,...,...,...
241,630602ac073e94de49556838,Sun Tzu,NaN,complet art war collect 4in1 omnibu import boo...
242,630602ac073e94de49556839,Disha Experts,NaN,free sampl ctet english hindi languag 9 yearwi...
243,630602ac073e94de4955683a,Yevgeny Zamyatin,NaN,yevgeni zamyatin set urban glass citi call one...
244,630602ac073e94de4955683b,Disha Experts,NaN,1 sampl paper cbse class 10 scienc 2020 exam none


In [50]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
main_df['author']= label_encoder.fit_transform(main_df['author'])
# main_df.drop("author", axis=1, inplace=True)
  
main_df.drop(main_df.index[main_df['_id'] == ObjectId("630602ac073e94de4955674a")], inplace=True)
main_df.drop(main_df.index[main_df['_id'] == ObjectId("630602ac073e94de49556749")], inplace=True)
main_df.drop(main_df.index[main_df['_id'] == ObjectId("630602ac073e94de49556747")], inplace=True)

# main_df

In [51]:
all_values = [{'_id': 0, 'author': 1000, "rate": 1, 'full_desc': ""},
{'_id': 0, 'author': 1000, "rate": 2, 'full_desc': ""},
{'_id': 0, 'author': 1000, "rate": 3, 'full_desc': ""},
{'_id': 0, 'author': 1000, "rate": 4, 'full_desc': ""},
{'_id': 0, 'author': 1000, "rate": 5, 'full_desc': ""}]

# all_values = [{'_id': 0, "rate": 1, 'full_desc': ""},
# {'_id': 0, "rate": 2, 'full_desc': ""},
# {'_id': 0, "rate": 3, 'full_desc': ""},
# {'_id': 0, "rate": 4, 'full_desc': ""},
# {'_id': 0, "rate": 5, 'full_desc': ""}]

main_df = main_df.append(all_values, ignore_index = True)

mask_var = main_df['rate'].isnull()  

main_df.sort_values('rate', inplace=True)
# display(main_df)
test_amount = len(main_df[mask_var])
print(test_amount)

238


C:\Users\User\AppData\Local\Temp\ipykernel_7844\1969007176.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(all_values, ignore_index = True)
C:\Users\User\AppData\Local\Temp\ipykernel_7844\1969007176.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_amount = len(main_df[mask_var])


In [52]:
main_df

,_id,author,rate,full_desc
243,0,1000,1.0,
1,630602ac073e94de4955674d,75,2.0,god war offici novel novel highli anticip god ...
244,0,1000,2.0,
3,630602ac073e94de49556781,62,3.0,game throne stori continu book 15 game throne ...
245,0,1000,3.0,
...,...,...,...,...
238,630602ac073e94de49556838,165,NaN,complet art war collect 4in1 omnibu import boo...
239,630602ac073e94de49556839,42,NaN,free sampl ctet english hindi languag 9 yearwi...
240,630602ac073e94de4955683a,179,NaN,yevgeni zamyatin set urban glass citi call one...
241,630602ac073e94de4955683b,42,NaN,1 sampl paper cbse class 10 scienc 2020 exam none


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

model = MultinomialNB()

def transform(text):
    cv_transf = CountVectorizer()
    tfidf_transf = TfidfTransformer()
    cv_transf.fit(text)
    cv_metrix = cv_transf.transform(text)
    # tfidf_transf.fit(cv_metrix)
    # result = tfidf_transf.transform(cv_metrix)
    return cv_metrix

In [54]:
from scipy.sparse import hstack
import numpy as np

data = transform(main_df["full_desc"])
data = hstack((data,np.array(main_df['author'])[:,None]))

In [55]:
from sklearn.tree import DecisionTreeClassifier

# model = DecisionTreeClassifier()
# model = MultinomialNB()
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB 
model = GaussianNB()

# split the entire dataset in train and test cuts: 50% and 50% respectively    
X_train, X_test, y_train, y_test = train_test_split(data, 
                                        main_df['rate'], test_size=test_amount,
                                        shuffle=False)

print(y_train)
print(y_test)

# train
trained_model = model.fit(X_train.toarray(), y_train)

# get all label predictions
pred = trained_model.predict(X_test.toarray())

pred

243    1.0
1      2.0
244    2.0
3      3.0
245    3.0
2      4.0
4      4.0
246    4.0
0      5.0
247    5.0
Name: rate, dtype: float64
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
       ..
238   NaN
239   NaN
240   NaN
241   NaN
242   NaN
Name: rate, Length: 238, dtype: float64


array([4., 3., 5., 3., 4., 4., 5., 5., 4., 3., 4., 5., 4., 5., 3., 4., 4.,
       5., 5., 4., 5., 3., 3., 4., 4., 4., 4., 5., 4., 3., 5., 5., 5., 3.,
       5., 4., 5., 4., 4., 5., 3., 4., 4., 3., 4., 5., 4., 4., 4., 3., 3.,
       5., 3., 3., 4., 3., 3., 5., 5., 3., 3., 4., 4., 4., 4., 5., 4., 3.,
       4., 3., 4., 4., 3., 4., 4., 4., 5., 4., 4., 3., 3., 4., 3., 5., 4.,
       3., 3., 4., 4., 4., 4., 4., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4.,
       3., 4., 4., 3., 3., 5., 4., 3., 4., 5., 5., 3., 3., 3., 2., 5., 2.,
       2., 5., 3., 4., 3., 3., 4., 3., 3., 4., 4., 4., 4., 4., 4., 3., 4.,
       4., 3., 3., 3., 5., 5., 4., 4., 3., 3., 4., 4., 4., 4., 3., 4., 3.,
       3., 5., 4., 3., 4., 3., 3., 3., 5., 4., 4., 3., 5., 5., 4., 5., 4.,
       4., 4., 4., 3., 3., 4., 3., 3., 5., 3., 5., 3., 4., 5., 3., 4., 3.,
       4., 4., 5., 3., 3., 4., 3., 5., 5., 5., 4., 5., 3., 3., 4., 5., 3.,
       2., 5., 4., 3., 3., 4., 4., 4., 4., 4., 4., 3., 5., 4., 3., 4., 5.,
       3., 4., 3., 4., 4.

In [ ]:
from sklearn import linear_model
from sklearn import tree
from sklearn.linear_model import LogisticRegression

# model = linear_model.BayesianRidge()
model = linear_model.LogisticRegression()
# model = linear_model.TweedieRegressor()
# model = tree.DecisionTreeRegressor()

# train
trained_model = model.fit(X_train.toarray(), y_train)

# get all label predictions
pred = trained_model.predict(X_test)

pred

In [ ]:
X_train.toarray()

In [ ]:
mask_var = main_df['rate'].isnull()  

# test_df = main_df[mask_var]  
# test_df

# train_df = main_df[~mask_var]  
# train_df

main_df.sort_values('rate', inplace=True)
main_df
test_amount = main_df[mask_var]  

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

model = MultinomialNB()

def transform(data):
    cv_transf = CountVectorizer()
    tfidf_transf = TfidfTransformer()
    cv_transf.fit(data)
    cv_metrix = cv_transf.transform(data)
    tfidf_transf.fit(cv_metrix)
    result = tfidf_transf.transform(cv_metrix)
    return result

In [ ]:
from scipy.sparse import hstack
import numpy as np

train_data = transform(train_df["full_desc"])
train_data = hstack((train_data,np.array(train_df['author'])[:,None]))
train_data.todense()

test_data = transform(test_df["full_desc"])
test_data = hstack((test_data,np.array(test_df['author'])[:,None]))
test_data.todense()

In [ ]:
# split the entire dataset in train and test cuts: 50% and 50% respectively    
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, 
                                        df1['sentiment'], test_size=test_amount)

# train
trained_model = model.fit(train_data, train_df["rate"])

# get all label predictions
pred = trained_model.predict(test_data)

pred

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X, y = load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
# gnb = GaussianNB()
# y_pred = gnb.fit(X_train, y_train).predict(X_test)
X